# End-to-End Freebase QA

Use the below line to import modules from the root of the repository.

Reference: https://stackoverflow.com/questions/34976803/sys-path-different-in-jupyter-and-python-how-to-import-own-modules-in-jupyter

In [5]:
import sys
sys.path.insert(0, '../../../allennlp')
sys.path.insert(0, '../../')

We use the cursor from psycopg2 to connect to PostgreSQL with our knowledge graph.

In [6]:
from scripts.utils.connect import get_connection 

connection = get_connection()
cursor = connection.cursor()

Load the Simple Questions V2 validation dataset.

In [7]:
from scripts.utils.simple_qa import load_simple_qa 

df_dev, = load_simple_qa(dev=True)
df_dev[:5]

,subject,relation,object,question
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...


For every example, there is not a clear subject name but we can do a fair job a linking questions with subject names only missing 2.7% of examples.

## Predict Subject Name

Using a tagging model, we are going to predict the subject name for every row.

In [9]:
from allennlp.models.archival import load_archive
from allennlp.service.predictors import Predictor
from scripts.utils.link_subject_name import tokenize


SUBJECT_RECOGNITION = '../../../allennlp/experiments/crf_tagger_lstm_no_reg_3/model.tar.gz'
archive = load_archive(SUBJECT_RECOGNITION, cuda_device=0)
predictor = Predictor.from_archive(archive, 'sentence-tagger')

def predict_subject_name(s):
    s =  ' '.join(tokenize(s))
    predicted = predictor.predict_json({'sentence': s}, 0)
    return ' '.join([predicted['words'][i] for i, tag in enumerate(predicted['tags']) if tag == 'I'])

predict_subject_name('what major cities does u.s. route 2 run through')

'u.s. route 2'

In [10]:
df_dev['predicted_subject_name'] = df_dev.apply(lambda row: predict_subject_name(row['question']), axis=1)
df_dev[:5]

,subject,relation,object,question,predicted_relations,predicted_subject_name
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...,{'symbols/namesake/named_after': 0.99915540142...,trump ocean club international hotel and tower
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born,{'people/person/place_of_birth': 0.98969678092...,sasha vujačić
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in,"{'music/release/region': 0.9990186741067519, '...",dead combo
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?,"{'film/director/film': 0.9050942595351116, 'fi...",wiebke von carolsfeld
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...,"{'music/release/region': 0.8322873397258621, '...",music for stock exchange


## Evaluate Predicted Subject Name

Here we compare the predicted subject name to the true subject name.

In [11]:
from scripts.utils.link_subject_name import add_subject_name

# Add True Subject Name
add_subject_name(df_dev, cursor, print_=True)

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))

Subject MID (0yzv6q0) does not have aliases.
Subject MID (0k571hg) does not have aliases.
Subject MID (02z6y3p) does not have aliases.
Subject MID (08vqg8n) does not have aliases.
Subject MID (045p9_r) does not have aliases.
Subject MID (03czw5g) does not have aliases.
Subject MID (04j2m25) does not have aliases.
Subject MID (0bvrjh4) does not have aliases.
Subject MID (08c_49) does not have aliases.
Subject MID (0h4867) does not have aliases.
Subject MID (02qplxv) does not have aliases.
Subject MID (0kz5hyk) does not have aliases.
Subject MID (07wk3m) does not have aliases.
Subject MID (01jyqwk) does not have aliases.
Subject MID (0_s2kgd) does not have aliases.
Subject MID (02vk_bv) does not have aliases.
Subject MID (03f86d) does not have aliases.
Subject MID (0kyvhwk) does not have aliases.
Subject MID (0l8d74) does not have aliases.
Subject MID (04jnfd) does not have aliases.
Subject MID (04zfl8) does not have aliases.
Subject MID (0z5y43n) does not have aliases.
Subject MID (040b

,Aliases,Question,Subject
0,"[bill hosket, jr.]","What was bill hosket, jr.'s position",02vvz00
1,"[megan leigh romero, megan romero]",What track is featured on the salisbury release,098j5p7
2,[datskat],What composer created?,0zhwd1v
3,[ron warner],Who's a linebacker,03m9zc3
4,"[mr m, mr. m., mr.m.]",what label does the artist mr. m. belong to,01x20gj
5,"[sabrina fredrica washington, sabrina washingt...",Which label is sabrinawmusic signed to?,0bnsl7
6,"[t-town, tulsa, tulsa, oklahoma, wagoner count...",What newspaper circulates in the town of kearny,013kcv
7,[nguyễn văn toàn],Where is the place of birth of nguyen van toan,02vnwfd
8,"[aftereight, capital lights]",what types of music is played by capitallights,02pvx1q
9,"[35 mm film, 35mm film]",what is an example of a film on 35mm,0cj16


### Numbers
2.351314% [255 of 10845] questions do not reference subject

0.461042% [50 of 10845] subject mids do not have aliases

In [12]:
from tqdm import tqdm_notebook
import random
from scripts.utils.link_subject_name import normalize_alias

random.seed(123)

correct = 0
incorrect_sample = []
for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    if isinstance(row['subject_name'], str):
        if normalize_alias(row['predicted_subject_name'].split()) == normalize_alias(row['subject_name']):
            correct += 1
        else:
            incorrect_sample.append([
                row['question'],
                normalize_alias(row['predicted_subject_name']),
                normalize_alias(row['subject_name'])
            ])

print('Accuracy:', correct / df_dev.shape[0])
print('Sample Incorrect:')
for i, items in enumerate(random.sample(incorrect_sample, 50)):
    print('| ' + str(i) + ' | '  + ' | '.join(items) + ' |')

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))


Accuracy: 0.9219917012448133
Sample Incorrect:
| 0 | what animated cartoon was about ducks? | duck | anim cartoon |
| 1 | which season of andrew zimmerman's show did he visit uganda in | andrew zimmerman | uganda |
| 2 | who is the most influential female k-pop singer? | singer | k-pop |
| 3 | What's an example of a novel film adaptation |  | film adapt |
| 4 | What is a city in Surrey County, north carolina? | surrey counti | north carolina |
| 5 | what kind of rock music is march ör die | march ör | march ör die |
| 6 | what is about between two women | two women | between two women |
| 7 | what genre is the 1962 the manchurian candidate film  | 1962 the manchurian candid | the manchurian candid |
| 8 | what the subject of the di medici bride  | di medici bride | the di medici bride |
| 9 | Name a netflix film that falls under the movies for ages 8 to 10 category | age 8 to 10 | movi for age 8 to 10 |
| 10 | What is a type of assassination in ways which appear natural |  | assassin 

| index | question | predicted | true | bucket |
|:--|:--|:--|:--|
| 0 | Who is the eighth avatar in hinduism  | eighth avatar | hinduism | wrong entity |
| 1 | what is a korean drama? | drama | korean drama | missing word left |
| 2 | Name a year the boston celtics won the NBA finals.  |  | boston celtic | no entity |
| 3 | what is a song from the there was an elk album  | elk | there was an elk | missing words left |
| 4 | what artists are included when discussing rock and roll | discuss rock and roll | rock and roll | extra word left |
| 5 | Which infectious disease is the vaccine dpt vaccine used to treat? | vaccin dpt vaccin | dpt vaccin | extra word left |
| 6 | which amusement park is the enchanted tiki room (under new management) in | the enchant tiki room | the enchant tiki room ( under new manag ) | missing words right |
| 7 | Name an indie album | indi album | indi | extra word right |
| 8 | Who is an artist  signed by rca records? | rca record | rca | extra word right |
| 9 | what does dubois county, indiana contain | duboi counti | duboi counti , indiana | missing words right |
| 10 | which comic book character has power of flight | power of flight | flight | extra words left |
| 11 | which country was the cosmic egg released in | the cosmic egg | cosmic egg | extra word left |
| 12 | Which bollywood film stars Menekka Arora and Arun Bali | arun bali | bollywood | wrong entity |
| 13 | which artist is known for creating  blues music | creat blue | blue music | |
| 14 | what's one event that celebrates the leather subculture | leather | leather subcultur |
| 15 | what film festival is recurring? | recur | film festiv |
| 16 | Where is the carrollton viaduct bridge located? | carrollton viaduct bridg | carrollton viaduct |
| 17 | what is a track name from the the corrs | the the corr | the corr |
| 18 | This mountain is the most easterly of the Munro peaks. | easter of the munro peak | munro |
| 19 | Which country was the hunyadi family from | the hunyadi famili | hunyadi famili |
| 20 | what is an example of a b movie? | b | b movi |
| 21 | What tv program utilized the バブルガムクライシス 6: red eyes soundtrack | バブルガムクライシス 6 : red eye soundtrack | バブルガムクライシス 6 : red eye |
| 22 | what language is sung in you don't know what love is song | sung in you do n't know what love is song | you do n't know what love is |
| 23 | what type of music is from a scream to a whisper | a scream to a whisper | from a scream to a whisper |
| 24 | where does the water in the canche come from | the water in the canch | canch |
| 25 | What type of release was in stereo | stereo | in stereo |
| 26 | where in austrailia is municipality of kiama the located in | kiama the | municip of kiama |
| 27 | The university of detroit mercy is what type of institution? | the univers of detroit merci | univers of detroit merci |
| 28 | what is the spoken language of the remembrance film | the remembr | remembr |
| 29 | who recorded the tower of london | the tower of london | tower of london |
| 30 | what kind of album is raffis christmas album | raffi christma | raffi christma album |
| 31 | what type of gallery is the cobra museum | the cobra museum | cobra museum |
| 32 | Who was killed by the electric chair for murder | the electr chair for murder | electr chair |
| 33 | whereis nirsa located | wherei nirsa | nirsa |
| 34 | who is a current member of  united states house committee on the judiciary | unit state hous committe on the judiciari | hous committe on the judiciari |
| 35 | what industry is sonoma wire works part of | sonoma wire | sonoma wire work |
| 36 | What is the name of a record producer? | produc | record produc |
| 37 | what is a type of cvg sports game | cvg sport game | sport game |
| 38 | who is the creator of are you smarter than a 5th grader? | are you smarter than a 5th grader | are you smarter than a 5th grader ? |
| 39 | where was mutaz kailouni bonr | mutaz kailouni bonr | mutaz kailouni |
| 40 | Name an album released by aaron carter | aaron carter | album |
| 41 | what type of music is of wars in osyrhia? | war in osyrhia | of war in osyrhia |
| 42 | Name a radisson hotels hotel | radisson hotel hotel | radisson hotel |
| 43 | what is a gay & lesbian romance found on netflix? | a gay & lesbian romanc | gay & lesbian romanc |
| 44 | which album is created by kylie minogue | album | kyli minogu |
| 45 | who wrote the lyrics for who feels love? | who feel love | who feel love ? |
| 46 | What type of music does krgy station play? | krgi station | krgi |
| 47 | which singer has native americans in the united states ethnicity  | unit state | nativ american in the unit state |
| 48 | What social science is the subject of the friendship factor | friendship factor | the friendship factor |
| 49 | the urviș river is contained by what country | the urviș river | urviș river |

### Index Normalized Alias for Linking

To help with linking, we index fb_name with normalized aliases.

In [ ]:
from tqdm import tqdm_notebook
import psycopg2
from scripts.utils.link_subject_name import normalize_alias


chunk_size = 10000

def update_chunk(rows):
    query = 'UPDATE fb_name SET normalized_alias = %s WHERE mid = %s and alias = %s'
    psycopg2.extras.execute_batch(cursor, query, rows)

cursor.execute('SELECT mid, alias FROM fb_name')
rows = []
for mid, alias in tqdm_notebook(cursor.fetchall()):
    normalized_alias = normalize_alias(alias)
    rows.append(tuple([normalized_alias, mid, alias]))
    
    
    # Insert Chunk
    if len(rows) > chunk_size:
        update_chunk(rows)
        rows = []
        
update_chunk(rows)

In [ ]:
connection.commit()

In [ ]:
cursor.execute('CREATE INDEX fb_name_normalized_alias_trgm ON fb_name USING gist(normalized_alias gist_trgm_ops);')
connection.commit()

### Link Predicted Subject Alias to MID

We try to handle the cases of missing words or extra words on the left or right by progressively removing and using ILIKE.

In [13]:
from tqdm import tqdm_notebook
from numpy import nan
import pandas as pd
import random

correct = 0
predicted_mids_column = []
incorrect = []
for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    predicted = normalize_alias(row['predicted_subject_name'].split())
    cursor.execute('SELECT mid FROM fb_name WHERE normalized_alias = %s', (predicted, ))
    predicted_mids = set([row[0] for row in cursor.fetchall()])

    if len(predicted_mids) == 0:
        # CASE: Missing words on left or right causing no matches
        cursor.execute("""
            SELECT mid, similarity(normalized_alias, %s) AS similarity_score 
            FROM fb_name
            WHERE normalized_alias %% %s
            ORDER BY similarity_score DESC
            LIMIT 10;
        """, (predicted, predicted))
        rows = list(cursor.fetchall())
        max_similarity_score = max(row[1] for row in rows)
        predicted_mids = set([r[0] for r in rows if r[1] == max_similarity_score])

        
    predicted_mids_column.append(predicted_mids)
    if row['subject'] in predicted_mids:
        correct += 1
    else:
        incorrect_sample.append([
            row['question'],
            predicted,
            normalize_alias(row['subject_name']),
        ])

print('Accuracy: %f [%d of %d]' % (correct / df_dev.shape[0], correct, df_dev.shape[0]))

# EXPERIMENT Best linking strategy ~
# normalized_alias + max similarity lookup: 0.618213 Expected Accuracy
# just normalized_alias lookup: 0.580540 Expected Accuracy
# normalized_alias + similarity lookup: 0.586586 Expected Accuracy
# normalized_alias + chopping off shortest word: 0.592118 Expected Accuracy
# normalized_alias + chopping off both words: 0.592412 Expected Accuracy

expected_accuracy = sum(1 / len(p) for p in predicted_mids_column if len(p) != 0) / df_dev.shape[0]
print('Expected Linking Accuracy: %f' % (expected_accuracy))
print('Sample Incorrect:')
for i, items in enumerate(random.sample(incorrect_sample, 50)):
    print('| ' + str(i) + ' | '  + ' | '.join(items) + ' |')
df_dev['predicted_mids'] = pd.Series(predicted_mids_column, index=df_dev.index)
df_dev[:5]

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))


Accuracy: 0.952882 [10334 of 10845]
Expected Linking Accuracy: 0.619009
Sample Incorrect:
| 0 | who created the bartender character in atlas shrugged | atlas shrug | bartend charact in atlas shrug |
| 1 | what's one of the topics discussed in an artist against the third reich | against the third reich | an artist against the third reich |
| 2 | what's a book published under the southern illinois university press imprint | southern illinoi univers press imprint | southern illinoi univers press |
| 3 | What is a Japanese animation series | japanes anim | anim |
| 4 | What is the name of Jack DeJohnrette's album? | jack dejohnrett | album |
| 5 | What is a compilation album by creedence clearwater revival | compil album | creedenc clearwat reviv |
| 6 | Name a netflix film that falls under the movies for ages 8 to 10 category | age 8 to 10 | movi for age 8 to 10 |
| 7 | What is a type of assassination in ways which appear natural |  | assassin in way which appear natur |
| 8 | What is th

,subject,relation,object,question,predicted_relations,predicted_subject_name,subject_name,subject_name_start_index,subject_name_stop_index,predicted_mids
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...,{'symbols/namesake/named_after': 0.99915540142...,trump ocean club international hotel and tower,trump ocean club international hotel and tower,3.0,10.0,{0f3xg_}
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born,{'people/person/place_of_birth': 0.98969678092...,sasha vujačić,sasha vujačić,2.0,4.0,{07f3jg}
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in,"{'music/release/region': 0.9990186741067519, '...",dead combo,dead combo,5.0,7.0,"{031j8nn, 0gql46}"
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?,"{'film/director/film': 0.9050942595351116, 'fi...",wiebke von carolsfeld,wiebke von carolsfeld,6.0,9.0,{0c1cyhd}
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...,"{'music/release/region': 0.8322873397258621, '...",music for stock exchange,music for stock exchange,3.0,7.0,{0fvhc0g}


In [14]:
from tqdm import tqdm_notebook
from numpy import nan
import pandas as pd

correct = 0
predicted_facts_column = []
for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    cursor.execute('SELECT object_mid, relation, subject_mid FROM fb_kg WHERE subject_mid = ANY(%s)', (list(row['predicted_mids']), ))
    predicted_facts = cursor.fetchall()
    predicted_facts_column.append(predicted_facts)
    predicted_object_mids = set([fact[0] for fact in predicted_facts])
    if row['object'] in predicted_object_mids:
        correct += 1

df_dev['predicted_facts'] = pd.Series(predicted_facts_column, index=df_dev.index)
print('Accuracy:', correct / df_dev.shape[0])
df_dev[:5]

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))


Accuracy: 0.9623789764868603


,subject,relation,object,question,predicted_relations,predicted_subject_name,subject_name,subject_name_start_index,subject_name_stop_index,predicted_mids,predicted_facts
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...,{'symbols/namesake/named_after': 0.99915540142...,trump ocean club international hotel and tower,trump ocean club international hotel and tower,3.0,10.0,{0f3xg_},"[(0fr_b, location/location/containedby, 0f3xg_..."
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born,{'people/person/place_of_birth': 0.98969678092...,sasha vujačić,sasha vujačić,2.0,4.0,{07f3jg},"[(0jmk7, sports/professional_sports_team/draft..."
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in,"{'music/release/region': 0.9990186741067519, '...",dead combo,dead combo,5.0,7.0,"{031j8nn, 0gql46}","[(07ssc, music/release/region, 031j8nn), (05r4..."
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?,"{'film/director/film': 0.9050942595351116, 'fi...",wiebke von carolsfeld,wiebke von carolsfeld,6.0,9.0,{0c1cyhd},"[(02zsn, people/person/gender, 0c1cyhd), (0345..."
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...,"{'music/release/region': 0.8322873397258621, '...",music for stock exchange,music for stock exchange,3.0,7.0,{0fvhc0g},"[(0mr41hm, music/release/track_list, 0fvhc0g),..."


In [22]:
RELATION_CLASSIFIER = '../../logs/0209.12-06_14:39:36.relation_classifier/12m_06d_14h_42m_42s.pt'

import pprint

from nltk.tokenize.treebank import TreebankWordTokenizer

from lib.checkpoint import Checkpoint

pretty_printer = pprint.PrettyPrinter(indent=2)
tokenizer = TreebankWordTokenizer() # Same tokenizer used during training

relation_classifier_predict = Checkpoint(checkpoint_path=RELATION_CLASSIFIER, device=0).predict

def get_relation(question, top_k=5):
    """ 
    Given a question return the predicate in the question using `RELATION_CLASSIFIER` model.
    
    Args:
        question (str)
    Returns:
        list of predicates and their confidence
    """
    predicted = list(relation_classifier_predict(question, top_k=top_k))
    return {class_.replace('www.freebase.com/', ''): confidence[0] for class_, confidence in predicted}

# To test this cell
print(get_relation('Where was Obama born?'))

{'people/person/place_of_birth': 0.9479034016893216, 'location/location/people_born_here': 0.037564982908731455, 'people/person/nationality': 0.00636750451682919, 'fictional_universe/fictional_character/place_of_birth': 0.00466165678268159, 'music/artist/origin': 0.0005357958454858299}


In [23]:
from tqdm import tqdm_notebook
from numpy import nan
import pandas as pd

correct = 0
predicted_relations_column = []
for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    predicted_relations = get_relation(row['question'], 10)
    predicted_relations_column.append(predicted_relations)
    if row['relation'] in set(predicted_relations.keys()):
        correct += 1
        
df_dev['predicted_relations'] = pd.Series(predicted_relations_column, index=df_dev.index)
print('Accuracy:', correct / df_dev.shape[0])
df_dev[:5]

# SOTA  0.973351775011526

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))

Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration




Accuracy: 0.973351775011526


,subject,relation,object,question,predicted_relations,predicted_subject_name,subject_name,subject_name_start_index,subject_name_stop_index,predicted_mids,predicted_facts
0,0f3xg_,symbols/namesake/named_after,0cqt90,Who was the trump ocean club international hot...,{'symbols/namesake/named_after': 0.99949658671...,trump ocean club international hotel and tower,trump ocean club international hotel and tower,3.0,10.0,{0f3xg_},"[(0fr_b, location/location/containedby, 0f3xg_..."
1,07f3jg,people/person/place_of_birth,0565d,where was sasha vujačić born,{'people/person/place_of_birth': 0.99203098224...,sasha vujačić,sasha vujačić,2.0,4.0,{07f3jg},"[(0jmk7, sports/professional_sports_team/draft..."
2,031j8nn,music/release/region,07ssc,What is a region that dead combo was released in,"{'music/release/region': 0.9913959852492691, '...",dead combo,dead combo,5.0,7.0,"{031j8nn, 0gql46}","[(07ssc, music/release/region, 031j8nn), (05r4..."
3,0c1cyhd,film/director/film,0wxsz5y,What is a film directed by wiebke von carolsfeld?,"{'film/director/film': 0.8895838289781793, 'fi...",wiebke von carolsfeld,wiebke von carolsfeld,6.0,9.0,{0c1cyhd},"[(02zsn, people/person/gender, 0c1cyhd), (0345..."
4,0fvhc0g,music/release/region,0345h,what country was music for stock exchange rel...,"{'music/release/region': 0.8462502052206398, '...",music for stock exchange,music for stock exchange,3.0,7.0,{0fvhc0g},"[(0mr41hm, music/release/track_list, 0fvhc0g),..."


In [33]:
from tqdm import tqdm_notebook
from numpy import nan
import pandas as pd
from collections import defaultdict

#  SOTA 0.797142
# Like Allenlp add accuracy with TQDM

correct = 0
relation_correct = 0
skipped = 0
total = 0
for index, row in tqdm_notebook(df_dev.iterrows(), total=df_dev.shape[0]):
    if len(row['predicted_facts']) > 0 and len(row['predicted_facts']) < 10000:
        max_relation_confidence = -1
        max_subject_mid = None
        max_relation = None
            
        for fact in row['predicted_facts']:
            _, relation, subject  = tuple(fact)
            if relation in row['predicted_relations']:
                relation_confidence = row['predicted_relations'][relation]
            else:
                relation_confidence = 0
            if relation_confidence > max_relation_confidence:
                max_relation_confidence = relation_confidence
                max_relation = relation
                max_subject_mid = subject
        
        # TODO: Group them up earlier so this is not as slow
        answers = set([f[0] for f in row['predicted_facts'] if f[2] == max_subject_mid and f[1] == max_relation])
        if row['object'] in answers:
            correct += 1
        if max_relation == row['relation']:
            relation_correct += 1
    else:
        skipped += 1
    total += 1

print('Skipped: %f [%d of %d]' % (skipped / total, skipped, total))
print('Accuracy: %f [%d of %d]' % (correct / total, correct, total))
print('Relation Accuracy: %f [%d of %d]' % (relation_correct / total, relation_correct, total))

HBox(children=(IntProgress(value=0, max=10845), HTML(value='')))


Skipped: 0.070355 [763 of 10845]
Accuracy: 0.750853 [8143 of 10845]
Relation Accuracy: 0.805901 [8740 of 10845]
